In [1]:
!pip install nsfw-detector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.16.1
    Uninstalling tensorflow-hub-0.16.1:
      Successfully uninstalled tensorflow-hub-0.16.1


In [4]:
from nsfw_detector import predict
model = predict.load_model('./nsfw_mobilenet2.224x224.h5')

# Predict single image
predict.classify(model, 'download.jpg')
# {'2.jpg': {'sexy': 4.3454722e-05, 'neutral': 0.00026579265, 'porn': 0.0007733492, 'hentai': 0.14751932, 'drawings': 0.85139805}}

ValueError: saved_model_path must be the valid directory of a saved model to load.

!pip install --upgrade tensorflow tensorflow_hub

In [5]:
import assemblyai as aai
import pyaudio

# Define handlers for transcription data and errors
def on_data(transcript: aai.RealtimeTranscript):
    if not transcript.text:
        return
    if isinstance(transcript, aai.RealtimeFinalTranscript):
        print(transcript.text, end="\r\n")
    else:
        print(transcript.text, end="\r")

def on_error(error: aai.RealtimeError):
    print("An error occurred:", error)

# Initialize the RealtimeTranscriber object
transcriber = aai.RealtimeTranscriber(
    on_data=on_data,
    on_error=on_error,
    sample_rate=44100,
)

# Connect to the WebSocket
transcriber.connect()

# Open microphone stream to capture audio
microphone_stream = aai.extras.MicrophoneStream()
transcriber.stream(microphone_stream)

# Close the WebSocket connection when done
transcriber.close()

ModuleNotFoundError: No module named 'assemblyai'

In [8]:
pip install pvcheetah

In [9]:
access_key = 'wJ0qa2Rl0HhisvuFnK7w9TZErHb5zb+bn/qU3yLOoDIlwRKFKbtXag=='
handle = pvcheetah.create(access_key)

NameError: name 'pvcheetah' is not defined

In [10]:
!pip install pvcheetah # Installs the pvcheetah package
import pvcheetah # Imports the package to be used in the code

access_key = 'wJ0qa2Rl0HhisvuFnK7w9TZErHb5zb+bn/qU3yLOoDIlwRKFKbtXag=='
handle = pvcheetah.create(access_key)

def get_next_audio_frame():
    pass

while True:
    partial_transcript, is_endpoint = handle.process(get_next_audio_frame())
    if is_endpoint:
        final_transcript = handle.flush()

In [16]:
import pvcheetah
import pyaudio
import numpy as np
import threading
import queue

class CheetahTranscriber:
    def __init__(self, access_key):
        """
        Initialize Cheetah transcriber

        :param access_key: Picovoice Cheetah access key
        """
        # Picovoice Cheetah setup
        self.access_key = access_key
        self.handle = pvcheetah.create(self.access_key)

        # Audio recording parameters
        self.CHUNK = 512  # Smaller chunk for real-time responsiveness
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 16000  # Cheetah typically expects 16kHz

        # PyAudio setup
        self.pyaudio = pyaudio.PyAudio()

        # Queues for audio and transcription
        self.audio_queue = queue.Queue()
        self.transcript_queue = queue.Queue()

        # Threading flags
        self.is_listening = False
        self.listen_thread = None
        self.transcribe_thread = None

    def _audio_capture_thread(self):
        """
        Continuously capture audio and put into queue
        """
        stream = self.pyaudio.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            frames_per_buffer=self.CHUNK
        )

        while self.is_listening:
            try:
                # Read audio chunk
                audio_chunk = stream.read(self.CHUNK)

                # Convert to numpy array
                audio_data = np.frombuffer(audio_chunk, dtype=np.int16)

                # Put into audio queue
                self.audio_queue.put(audio_data)

            except Exception as e:
                print(f"Audio capture error: {e}")
                break

        stream.stop_stream()
        stream.close()

    def _transcription_thread(self):
        """
        Process audio chunks and generate transcriptions
        """
        while self.is_listening:
            try:
                # Get audio frame from queue
                audio_frame = self.audio_queue.get(timeout=1)

                # Process with Cheetah
                partial_transcript, is_endpoint = self.handle.process(audio_frame)

                # If endpoint reached, get final transcript
                if is_endpoint:
                    final_transcript = self.handle.flush()

                    # Add to transcript queue if not empty
                    if final_transcript and final_transcript.strip():
                        self.transcript_queue.put(final_transcript)
                        print(f"Final Transcript: {final_transcript}")

                # Add partial transcripts if available
                if partial_transcript and partial_transcript.strip():
                    print(f"Partial Transcript: {partial_transcript}")

            except queue.Empty:
                continue
            except Exception as e:
                print(f"Transcription error: {e}")

    def start_transcription(self):
        """
        Start audio capture and transcription
        """
        # Reset handle to ensure clean state
        self.handle = pvcheetah.create(self.access_key)

        # Set listening flag
        self.is_listening = True

        # Start audio capture thread
        self.listen_thread = threading.Thread(target=self._audio_capture_thread)
        self.listen_thread.start()

        # Start transcription thread
        self.transcribe_thread = threading.Thread(target=self._transcription_thread)
        self.transcribe_thread.start()

        print("Transcription started. Speak into the microphone.")

    def stop_transcription(self):
        """
        Stop audio capture and transcription
        """
        # Set flag to stop threads
        self.is_listening = False

        # Wait for threads to finish
        if self.listen_thread:
            self.listen_thread.join()
        if self.transcribe_thread:
            self.transcribe_thread.join()

        # Close PyAudio
        self.pyaudio.terminate()

        # Clean up Cheetah handle
        self.handle.delete()

        print("Transcription stopped.")

    def get_transcripts(self):
        """
        Retrieve transcripts from queue

        :return: List of transcripts
        """
        transcripts = []
        while not self.transcript_queue.empty():
            transcripts.append(self.transcript_queue.get())
        return transcripts

def main():
    # Replace with your actual Picovoice Cheetah access key
    ACCESS_KEY = access_key

    try:
        # Create transcriber
        transcriber = CheetahTranscriber(ACCESS_KEY)

        # Start transcription
        transcriber.start_transcription()

        # Keep main thread running
        while True:
            import time
            time.sleep(5)

            # Check and print transcripts periodically
            transcripts = transcriber.get_transcripts()
            if transcripts:
                print("Recent Transcripts:", transcripts)

    except KeyboardInterrupt:
        print("\nStopping transcription...")

    finally:
        # Ensure transcription stops
        transcriber.stop_transcription()

if __name__ == "__main__":
    main()

Exception in thread Thread-11 (_audio_capture_thread):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-16-c96ea4731dbc>", line 40, in _audio_capture_thread
  File "/usr/local/lib/python3.10/dist-packages/pyaudio/__init__.py", line 639, in open
    stream = PyAudio.Stream(self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyaudio/__init__.py", line 441, in __init__
    self._stream = pa.open(**arguments)
OSError: [Errno -9996] Invalid input device (no default output device)


Transcription started. Speak into the microphone.

Stopping transcription...
Transcription stopped.


In [14]:
!pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [15]:
!apt-get update
!apt-get install -y portaudio19-dev libportaudio2 libportaudiocpp0
!pip install pyaudio

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,515 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,513 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages 

TypeError: object of type 'NoneType' has no len()